In [1]:
import os
os.environ['JAX_PLATFORMS'] = ''

from os import listdir
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from cv2 import imread, imshow, resize

In [2]:
base_path = "E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & UNG DUNG/BREAST CANCER/data/IDC_regular_ps50_idx5"
folder = listdir(base_path)
len(folder)
print(folder[:5])

['10253', '10254', '10255', '10256', '10257']


In [3]:
# how many image input?
total_images = 0
for n in range(len(folder)):
    patient_id = folder[n]
    for c in [0, 1]:
        patient_path = base_path + "/" + patient_id
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        total_images += len(subfiles)

print(total_images)

277524


In [4]:
# split patient into frame: patient_id, path, target (id patient, path/to/patient and label)
data = pd.DataFrame(index=np.arange(0, total_images), columns=["patient_id", "path", "target"])

k = 0
for n in range(len(folder)):
    patient_id = folder[n]
    patient_path = base_path + "/" + patient_id 
    for c in [0,1]:
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        for m in range(len(subfiles)):
            image_path = subfiles[m]
            data.loc[k, "path"] = class_path + image_path
            data.loc[k, "target"] = c
            data.loc[k, "patient_id"] = patient_id
            k += 1  

data.head()

,patient_id,path,target
0,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0
1,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0
2,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0
3,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0
4,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0


In [5]:
Pos_path = data.groupby("target").get_group(0)
Neg_path = data.groupby('target').get_group(1)

In [6]:
print(Neg_path[0:5])
print(len(Neg_path))

    patient_id                                               path target
479      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
480      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
481      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
482      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
483      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
78786


In [7]:
lowerIndex = 0
upperIndex = 50000

Pos_train_path = list(Pos_path[lowerIndex: upperIndex]['path'])
Pos_train_target = list(Pos_path[lowerIndex: upperIndex]['target'])

print(len(Pos_train_path))
print(len(Pos_train_target))


50000
50000


In [8]:
Neg_train_path = list(Neg_path[lowerIndex: upperIndex]['path'])
Neg_train_target = list(Neg_path[lowerIndex: upperIndex]['target'])

print(len(Pos_train_path))
print(len(Neg_train_target))

50000
50000


In [9]:
x_train = []
y_train = []
x_test = []
y_test = []

WIDTH = 50
HEIGHT = 50

for path in Pos_train_path:
    origImageSize = imread(path)
    resizedImage = resize(origImageSize, (WIDTH, HEIGHT), interpolation=cv2.INTER_CUBIC)
    x_train.append(resizedImage)

for path in Neg_train_path:
    origImageSize = imread(path)
    resizedImage = resize(origImageSize, (WIDTH, HEIGHT), interpolation=cv2.INTER_CUBIC)
    x_train.append(resizedImage)

In [10]:
print(len(x_train))
y_train = Pos_train_target + Neg_train_target
print(len(y_train))

100000
100000


In [11]:
combined = list(zip(x_train, y_train))
np.random.shuffle(combined)
x_train, y_train = zip(*combined)

In [12]:
x_train = np.array(x_train)/255.0
y_train = np.array(y_train)

In [13]:
print(x_train.shape)
print(y_train.shape)

(100000, 50, 50, 3)
(100000,)


***note***: nếu gặp lỗi như thế này: RuntimeError: Unable to initialize backend 'tpu': UNIMPLEMENTED: LoadPjrtPlugin is not implemented on windows yet. (set JAX_PLATFORMS='' to automatically choose an available backend)

=> phải đảm bảo rằng thiết lập biến môi trường trước khi import các module của keras. tensorflow...

=> để thiết lập biến môi trường thì thêm mã sau:

import os

os.environ['JAX_PLATFORMS'] = ''

In [14]:
# import os
# os.environ['JAX_PLATFORMS'] = ''

In [15]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, GlobalAveragePooling2D, Dropout
from keras.applications import MobileNet
from keras.callbacks import ModelCheckpoint

# MobileNet v1

In [16]:
def create_MobileNet(inputShape=(50, 50, 3), numClasses=1):
    # Load pre-trained model
    baseModel = MobileNet(weights=None, include_top=False, input_shape=inputShape)

    # add custom top layers for classification
    x = baseModel.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)

    predictions = Dense(numClasses, activation='sigmoid')(x)

    # combine base model with custom top layers
    model = Model(inputs=baseModel.input, outputs=predictions)

    return model

In [17]:
model = create_MobileNet(inputShape=(50, 50, 3), numClasses=1)
# model.summary()
callBack = ModelCheckpoint(
    filepath = "BreastCancerModel_MobileNet_v1.keras",
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True,
)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_v1 = model.fit(x_train, y_train, epochs=30, batch_size=64, validation_split=0.2, callbacks=[callBack])

MemoryError: Unable to allocate 2.79 GiB for an array with shape (100000, 50, 50, 3) and data type float32

In [ ]:
plt.plot(history_v1.history['accuracy'])
plt.plot(history_v1.history['val_accuracy'])
plt.title('Loss and Validation loss traning (MobileNet model)')
plt.xlabel('Epochs')
plt.ylabel('(%)')
plt.legend(['Accuracy', 'Val_accuracy'])
plt.show()

# MobileNet v2

In [ ]:
def create_MobileNet(inputShape=(50, 50, 3), numClasses=2):
    # Load pre-trained model
    baseModel = MobileNet(weights=None, include_top=False, input_shape=inputShape)

    # add custom top layers for classification
    x = baseModel.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.6)(x)

    predictions = Dense(numClasses, activation='softmax')(x)

    # combine base model with custom top layers
    model = Model(inputs=baseModel.input, outputs=predictions)

    return model

In [ ]:
model = create_MobileNet(inputShape=(50, 50, 3), numClasses=2)
model.summary()

sự thay đổi trong phiên bản lần này gồm:

    - thay đổi hàm tối ưu: sgd
    - thay đổi hàm mất mát: sparse_categorical_crossentropy
    - thay đổi số lớp là: 2
    - thay đổi hàm kích hoạt đầu ra là: softmax

In [ ]:
callBack = ModelCheckpoint(
    filepath = "BreastCancerModel_MobileNet_v2.keras",
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True,
)
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[callBack])